In [1]:
Dataset = "FoCus"                      # "Blended Skill Talk", "IT-ConvAI2" , "FoCus" 

_COT = ""                                   #  "",    "-COT"

SCORING_METHOD = "benchmark"                      # "avg",  "length_prior", "benchmark"

LLM_name = f"Qwen2-5B-Instruct-{SCORING_METHOD}"

In [2]:
LLM_name

'Qwen2-5B-Instruct-benchmark'

In [3]:
import pandas as pd

ds = pd.read_csv(f'Raw Responses/{Dataset}/{LLM_name}{_COT}.csv')

print("Shape:", ds.shape)

print("\nMissing Values:\n", ds.isnull().sum())

ds

Shape: (4000, 2)

Missing Values:
 gen_response     1536
response_time       0
dtype: int64


,gen_response,response_time
0,Hello! Welcome to Descent of the Ganges. It's ...,0.791476
1,The name of the place is Little Rock Central H...,0.757963
2,NaN,0.893476
3,Camp Randall Stadium is an outdoor stadium loc...,0.740705
4,NaN,0.894334
...,...,...
3995,NaN,0.876975
3996,Kõpu Lighthouse is a beautiful historical land...,0.667874
3997,St. Matthews Lutheran Church is a historic chu...,0.820739
3998,The bell has been restored to its original pos...,0.335340


In [4]:
print(ds.iloc[167]["gen_response"])

nan


In [6]:
import re
import json
import pandas as pd

# Define functions
def find_first_valid_json(text):

    if not isinstance(text, str):
        return None


    json_objects = re.findall(r'\{.*?\}', text, re.DOTALL)
    for obj in json_objects:
        try:
            json_obj = json.loads(obj)
            if "response" in json_obj:  # Only check for "response"
                return json_obj
        except json.JSONDecodeError:
            continue
    return None

def get_response(text):
    if text is not None:  # Check if text is not None
        try:
            return text['response']
        except (ValueError, SyntaxError, KeyError):
            return None
    return None



# Replace empty strings in 'gen_response' with None
ds.loc[ds['gen_response'] == '', 'gen_response'] = None

# Apply the find_first_valid_json function
ds['gen_response'] = ds['gen_response'].apply(lambda x: find_first_valid_json(x))

# Convert gen_response to None if it's not a valid string
ds['gen_response'] = ds['gen_response'].apply(lambda x: None if pd.isna(x) or x == 'nan' or isinstance(x, float) else x)

# Extract 'response' from the JSON objects
ds['gen_response'] = ds['gen_response'].apply(lambda x: get_response(x))

# Keep the 'response_time' column unchanged
ds['response_time'] = ds['response_time']

# Define the new column order
new_column_order = ['gen_response', 'response_time']

# Reorder the columns
ds = ds[new_column_order]

print("\nMissing Values:\n", ds.isnull().sum())
print(ds.shape)
ds



Missing Values:
 gen_response     4000
response_time       0
dtype: int64
(4000, 2)


,gen_response,response_time
0,None,0.791476
1,None,0.757963
2,None,0.893476
3,None,0.740705
4,None,0.894334
...,...,...
3995,None,0.876975
3996,None,0.667874
3997,None,0.820739
3998,None,0.335340


In [37]:
ds.to_csv(f'Responses/{Dataset}/{LLM_name}{_COT}.csv', index=False)